In [1]:
import numpy as np
import pandas as pd

In [92]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Bicycle_Rental_Prediction/train.csv', parse_dates=['date_time'], infer_datetime_format=True)
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Bicycle_Rental_Prediction/test.csv', parse_dates=['date_time'], infer_datetime_format=True)
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Bicycle_Rental_Prediction/sample_submission.csv')

In [93]:
all_data = pd.concat([train, test], ignore_index=True)
all_data_index = all_data.index

In [94]:
all_data.describe()

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
count,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,273.000000
mean,198.634725,2.319503,0.121179,2.383970,58.386926,13.891959,23.356146,17.328552,59574.978022
std,56.845031,0.976251,0.230958,0.868715,12.396575,4.980942,4.969229,16.315020,27659.575774
min,44.123000,1.000000,0.000000,0.753000,24.831000,1.938000,9.895000,0.000000,1037.000000
25%,168.287250,1.405000,0.000000,1.743500,48.809000,10.172250,19.895000,4.357750,36761.000000
50%,205.729500,2.196000,0.000000,2.310500,58.784000,14.500000,24.000000,13.209500,63032.000000
75%,234.637250,3.143750,0.142000,2.800000,68.446000,18.000000,27.526000,25.608500,81515.000000
max,321.622000,4.000000,1.514000,5.607000,88.885000,22.312000,33.421000,82.162000,110377.000000


In [95]:
na_check = all_data.isna().sum() / len(all_data)
na_check

date_time                    0.00
wind_direction               0.00
sky_condition                0.00
precipitation_form           0.00
wind_speed                   0.00
humidity                     0.00
low_temp                     0.00
high_temp                    0.00
Precipitation_Probability    0.00
number_of_rentals            0.25
dtype: float64

In [96]:
for i in range(1, 9):
  all_data['wind_direc_'+(str)(i)] = False

for i in range(1, 9):
  all_data.loc[(all_data['wind_direction'] >= (45*(i-1))) & (all_data['wind_direction'] < (45*i)), 'wind_direc_'+(str)(i)] = True

In [42]:
all_data = all_data.drop(['wind_direction'], axis=1)

In [97]:
all_data['daily_temp_range'] = all_data['high_temp'] - all_data['low_temp']

In [98]:
all_data['average_temp'] = (all_data['high_temp'] + all_data['low_temp']) / 2

In [99]:
col_l = ['sky_condition', 'precipitation_form', 'wind_speed',
       'humidity', 'low_temp', 'high_temp', 'Precipitation_Probability', 'daily_temp_range']
col_r = all_data.columns.drop(col_l)

In [100]:
left_data = all_data[col_l]

In [101]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(all_data[col_l])
left_data = scaler.transform(all_data[col_l])
left_data = pd.DataFrame(left_data, columns=col_l, index=all_data_index)
left_data.describe()

,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,daily_temp_range
count,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000
mean,0.439834,0.080039,0.336005,0.523869,0.586726,0.572182,0.210907,0.576087
std,0.325417,0.152548,0.178969,0.193533,0.244475,0.211223,0.198571,0.188621
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.135000,0.000000,0.204059,0.374340,0.404155,0.425062,0.053039,0.465181
50%,0.398667,0.000000,0.320869,0.530068,0.616570,0.599549,0.160774,0.578260
75%,0.714583,0.093791,0.421714,0.680910,0.788358,0.749426,0.311683,0.708988
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [102]:
right_data = all_data[col_r]

right_data['year'] = right_data['date_time'].dt.year
right_data['month'] = right_data['date_time'].dt.month
right_data['day'] = right_data['date_time'].dt.day
right_data['wDay'] = right_data['date_time'].dt.weekday
right_data.index = all_data_index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [103]:
all_data_ = pd.merge(left_data, right_data, left_index=True, right_index=True)#, on='id', how='outer')
#all_data_ = left_data.join(right_data, how='inner')

In [104]:
all_data = all_data_

In [105]:
all_data['YeMoWd'] = (pd.to_numeric(all_data['year'])*1000)+(pd.to_numeric(all_data['month'])*10)+all_data['wDay']

In [106]:
nTrain = len(train)
train = all_data[:nTrain]
test = all_data[nTrain:]

In [72]:
all_data.columns

Index(['sky_condition', 'precipitation_form', 'wind_speed', 'humidity',
       'low_temp', 'high_temp', 'Precipitation_Probability',
       'daily_temp_range', 'date_time', 'wind_direction', 'number_of_rentals',
       'wind_direc_1', 'wind_direc_2', 'wind_direc_3', 'wind_direc_4',
       'wind_direc_5', 'wind_direc_6', 'wind_direc_7', 'wind_direc_8',
       'average_temp', 'year', 'month', 'day', 'wDay'],
      dtype='object')

In [112]:
cormat = all_data.corr()
cormat = cormat.index[abs(cormat['number_of_rentals']) > 0.1]
cormat

Index(['sky_condition', 'precipitation_form', 'wind_speed', 'humidity',
       'low_temp', 'high_temp', 'Precipitation_Probability',
       'daily_temp_range', 'wind_direction', 'number_of_rentals',
       'wind_direc_2', 'wind_direc_3', 'wind_direc_4', 'wind_direc_5',
       'wind_direc_6', 'average_temp', 'year', 'month', 'YeMoWd'],
      dtype='object')

In [147]:
col_x = ['sky_condition', 'precipitation_form', 'wind_speed', 'humidity',
       'Precipitation_Probability', 'average_temp', 'YeMoWd']
col_y = ['number_of_rentals']

In [148]:
from sklearn.model_selection import train_test_split

train_x = train[col_x]
train_y = np.log1p(train['number_of_rentals'])

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True)

test_id_idx = test.index

test_x = test[col_x]

In [149]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state = 1)
xgb.fit(train_x, train_y)

[07:20:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=1,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [150]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
pred_train = xgb.predict(train_x)
pred_val = xgb.predict(val_x)

print('train mae score: ', mean_squared_error(train_y, pred_train))
print('val mae score:', mean_squared_error(val_y, pred_val))

# train mae score : 0.06544029142977467
# val mae score : 0.11464889851539106
# real score : 0.7983113151

# train mae score:  0.0697602397088063
# val mae score: 0.10759365201564267

train mae score:  0.007751978156959145
val mae score: 0.15287299797381826


In [151]:
pred = xgb.predict(test_x)

In [152]:
submission[col_y] = np.expm1(pred).astype(int)

In [153]:
submission

,date_time,number_of_rentals
0,2021-04-01,83945
1,2021-04-02,73119
2,2021-04-03,49941
3,2021-04-04,49219
4,2021-04-05,97794
...,...,...
86,2021-06-26,93903
87,2021-06-27,99996
88,2021-06-28,17633
89,2021-06-29,56704


In [154]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/Bicycle_Rental_Prediction/submission.csv', index=False)